# week 8

## goals:
- 

## todo
- 


## notes:

- specifically try to answer:
    - "you might want to just focus on MPAs that are beyond the continental shelves but we can talk about that."
        - focus on high seas
    - Should I include things like Ross Sea (which probably have a terrestrial component as well)?
    - How do I use the post-2016 data? I'm in the google [permission?] group – can I make queries against an API?
        - offline meeting with Dan to pull this data

- GFW Data Training <- this is the google group to get added to


- explicit theories about how they should behave and testing that

- Ross Sea
- PIPA
- PMNM
- south/central pacific ocean
- pick out the 5 from Tim White's paper:
    - PIPA - 2010
    - Pacific Remote Islands - 2009
    - PMNM - yes but low fishing effort (3258 hours, 2016)
    - Pitcairn Islands - yes but low (101 hours, 2016)
    - Nazca-Desventuradas - yes but low (68 hours, 2016)
    
Actual good ones:
- Natural Park of the Coral Seas, 2014 555577562, further off the east coast of Australia
- Marae Moana/Cook Islands, 2017 555624907 - maybe the best one - middle of the Pacific
- French Austral Lands and Seas, 2019 555697868 - south-east of Africa
- Arquipélago Submarino Do Meteor, 2016 555514087 - Atlantic ocean Y shaped area
- Coral Sea, 2018 555556875 just off the east coast of Australia
- Pacífico Mexicano Profundo, 2018 555624307 southwest Mexican coast
- Palau National Marine Sanctuary, 2015, 555622118 oceania (near indonesia, phillippines) 


Can't use:
- Phoenix Island - it's 2010


A bunch of these at the top are coastal around Spain/France:
- Espacio marino del Delta de l'Ebre-Illes Columbretes 2014
- Espacio marino de la Costa da Morte 2014
- Pertuis charentais - Rochebonne 2019




In [1]:
cd /Users/brendan/Masters/gfw_research

/Users/brendan/Masters/gfw_research


In [2]:
import numpy as np
import pandas
from datetime import datetime
import matplotlib.pyplot as plt
import os
import pywdpa
import geopandas
import contextily as ctx
from shapely import geometry
import pretty_html_table

import util 

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)

In [ ]:
## REUSABLE FUNCTIONS

def join_on_lat_lon(points):
    return points.groupby(['lat_bin', 'lon_bin'], as_index=False).aggregate({'lat_bin': 'first', 'lon_bin': 'first', 'fishing_hours': 'sum'})


## Compute geometries for the points
def compute_box(row, width=0.1):
    x = float(row['lon_bin'] * width)
    y = float(row['lat_bin'] * width)
    return geometry.box(x, y, x+width, y+width)


def convert_to_geo(points, box=True):
    if box:
        points['geometry'] = points.apply(compute_box, axis=1)
        return geopandas.GeoDataFrame(points, geometry=points['geometry']).set_crs(epsg=4326)
    # use points (not as precise, but cheaper)
    else:
        return geopandas.GeoDataFrame(
            points,
            geometry=geopandas.points_from_xy(points['lon_bin']*0.1 + 0.05, points['lat_bin']*0.1 + 0.05)
        ).set_crs(epsg=4326)

    
def load_mpas():
    # reads the downloaded WPDA polygon files
    filenames = [
        "../data/WDPA_WDOECM_wdpa_shp/WDPA_WDOECM_wdpa_shp0/WDPA_WDOECM_wdpa_shp-polygons.shp",
        "../data/WDPA_WDOECM_wdpa_shp/WDPA_WDOECM_wdpa_shp1/WDPA_WDOECM_wdpa_shp-polygons.shp",
        "../data/WDPA_WDOECM_wdpa_shp/WDPA_WDOECM_wdpa_shp2/WDPA_WDOECM_wdpa_shp-polygons.shp",
    ]
    protected_areas = []
    counted = 0
    for filename in filenames:
        print(f'\rloading mpas: {counted}/{len(filenames)}', end='')
        protected_areas.append(geopandas.read_file(filename))
        counted += 1
    print(f'\rloading mpas: {counted}/{len(filenames)} done.', end='')


    protected_areas = pandas.concat(protected_areas)
    # filters for marine only (may want to change this to 1 or 2 (2 is marine only, 1 is mixed, 0 is terrestrial))
    mpas = protected_areas[protected_areas["MARINE"] == "2"]
    return mpas


def load_points():
    ## Load the fishing hours data (this is kinda slow)
    print('loading points')
    filenames = os.listdir('../data/daily_csvs')

    # this might be faster but the status printout is nice:
    # points = pandas.concat([geopandas.read_file('data/daily_csvs/' + filename) for filename in filenames])

    counted = 0
    points = []
    for filename in filenames:
        print(f'\r {filename} {counted}/{len(filenames)}', end='')
        points.append(pandas.read_csv('../data/daily_csvs/' + filename,
                                          dtype={'lat_bin': 'int16',
                                                 'lon_bin': 'int16',
                                                 'mmsi': 'int32',
                                                 'fishing_hours': 'float32'},
                                     parse_dates=['date']))
        counted += 1
    print('\nloaded.')
    return pandas.concat(points) # deliberately overwriting points


In [3]:
# sample the points here (100 of them) before converting to geo
# use the sampled points to find the mmsi that are interesting
# only then should I get all of the points of relevent mmsi

mpas = util.load_mpas()
full_points = util.load_points()
SAMPLE_RATIO = 100
points_sampled = full_points.sample(frac=1/SAMPLE_RATIO)
geopoints_sampled = util.convert_to_geo(points_sampled, box=True)



loading mpas: 3/3 done.
loading points
 2013-12-20.csv 1826/1827
loaded.


In [ ]:
def table_of_in_out_pre_post(points_of_mpa_ships, mpa, date):
    """
    Returns a table with total fishing hours in/out of the mpa
    
    All it does is compute for each point whether it's in/out and before/after, and
    then aggregates the totals per combination for each mmsi (each unique ship).
    
    Tries to take advantage of table groupby and aggregation to do this.
    """
    geopoints_ = convert_to_geo(points_of_mpa_ships, box=True)
    geopoints_['pre'] = geopoints_['date'] < date
    # fastest way to do this is just this whole join:
    print('running sjoin on geopoints of mpa ships')
    geopoints_['in_mpa'] = ~geopandas.sjoin(geopoints_, mpa[['geometry']], how='left', op='within')['index_right'].isnull()
    # TODO - what about on the border/indeterminate?
    # geopoints_['intersects_mpa'] = ~geopandas.sjoin(geopoints_, mpa[['geometry']], how='left', op='intersects')['index_right'].isnull()
    aggregated_ = geopoints_.groupby(['mmsi', 'in_mpa', 'pre'], as_index=False).aggregate({'fishing_hours': 'sum', 'mmsi': 'first'})
    aggregated_['in_pre'] = aggregated_.apply(lambda row: row['fishing_hours'] if (row['in_mpa'] and row['pre']) else 0.0, axis=1)
    aggregated_['out_pre'] = aggregated_.apply(lambda row: row['fishing_hours'] if (not row['in_mpa'] and row['pre']) else 0.0, axis=1)
    aggregated_['in_post'] = aggregated_.apply(lambda row: row['fishing_hours'] if (row['in_mpa'] and not row['pre']) else 0.0, axis=1)
    aggregated_['out_post'] = aggregated_.apply(lambda row: row['fishing_hours'] if (not row['in_mpa'] and not row['pre']) else 0.0, axis=1)
    table = aggregated_.groupby(['mmsi'], as_index=False).aggregate({'in_pre': 'sum', 'out_pre': 'sum', 'in_post': 'sum', 'out_post': 'sum'})
    table['pre_percent_in'] = table['in_pre'] / (table['in_pre'] + table['out_pre'])
    table['post_percent_in'] = table['in_post'] / (table['in_post'] + table['out_post'])
    return table

   

def plot_effort_with_world(effort, mpa, linewidth=0.5, title=''):
    world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
    ax = world.plot(color='white', edgecolor='black', figsize=(20, 30))
    effort.plot(column='fishing_hours', cmap='Blues', scheme='quantiles', ax=ax, legend=True)
    mpa.plot(ax=ax, color='None', edgecolor='red', linewidth=linewidth, alpha=0.5)
    plt.title(title)


def analyze_mpa(geopoints_sampled, full_points, mpa, date, plot_pre_post=True):
    #mpa = mpas[mpas['WDPAID'] == 555577562.0]

    print('running sjoin... ', end='')
    mpa_points = geopandas.sjoin(geopoints_sampled, mpa, op='within')
    print('\nfound {} sampled points in the mpa from {} ships'.format(
        mpa_points.shape[0], mpa_points['mmsi'].nunique()))

    points_of_mpa_ships = geopandas.GeoDataFrame(full_points[full_points['mmsi'].isin(mpa_points['mmsi'])])

    print('found {} points of mpa ships'.format(points_of_mpa_ships.shape[0]))
    
    table = table_of_in_out_pre_post(points_of_mpa_ships, mpa, date,).sort_values('in_pre', ascending=False)
    
    pre = convert_to_geo(
        join_on_lat_lon(points_of_mpa_ships[points_of_mpa_ships['date'] < date]), box=True)
    post = convert_to_geo(
        join_on_lat_lon(points_of_mpa_ships[points_of_mpa_ships['date'] >= date]), box=True)
    
    if plot_pre_post:
        plot_effort_with_world(pre, mpa, title='effort pre-closure')
        plot_effort_with_world(post, mpa, title='effort post-closure')
 
    return table, pre, post
 

In [ ]:
coral_park = mpas[mpas['WDPAID'] == 555577562.0]
coral_park_date = '2014-05-28'
coral_park_results = util.analyze_mpa(geopoints_sampled, full_points, coral_park, coral_park_date)
coral_park_results[0]

running sjoin... 

In [ ]:
# with 1/100 sampling
pipa = mpas[mpas['WDPAID'] == 555512002.0]
pipa_date = '2015-01-01'
pipa_results = analyze_mpa(geopoints_sampled, full_points, pipa, pipa_date)
pipa_results[0]

In [ ]:
# with 1/500 sampling
pipa = mpas[mpas['WDPAID'] == 555512002.0]
pipa_date = '2015-01-01'
pipa_results = analyze_mpa(geopoints_sampled, full_points, pipa, pipa_date, plot_pre_post=False)
pipa_results[0]

In [ ]:
# with 1/1000 sampling
pipa = mpas[mpas['WDPAID'] == 555512002.0]
pipa_date = '2015-01-01'
pipa_results = analyze_mpa(geopoints_sampled, full_points, pipa, pipa_date)
pipa_results[0]